<a href="https://colab.research.google.com/github/gonzalobarrazueta/TF-20181e098-20191a972-201920156/blob/main/MAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Imports**

In [ ]:
# Perlin Noise
!pip install perlin_noise
import matplotlib.pyplot as plt
import numpy as np
from perlin_noise import PerlinNoise
# ------------
import csv
import json
# Sección Aristas
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
import random
# ------------

##**Diccionario de calles**
Crea un diccionario donde cada nombre de calle ***(key)*** tiene asignado un `id` ***(value)***

In [2]:
def addKey_Values(dictionary, key, value):
    dictionary[key] = value

In [115]:
def printDictionary(dictionary):
    for k, v in dictionary.items():
        print(f"{k}: {v}")

In [4]:
def cargarDatos(dictionary):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        
        for id, row in enumerate(reader):
            addKey_Values(dictionary, row[0].lower(), id)

In [118]:
def createStreetsDictionary():   
    streets = {}
    cargarDatos(streets)
    return streets

In [117]:
streetsDictionary = createStreetsDictionary()

##**Lista de Adyacencia de calles y sus intersecciones**




In [8]:
def printAdList(graph): 
    for i, intersection in enumerate(graph):
        print(i, intersection)
    
    print(f"Cantidad de vértices = {len(graph)}")

In [7]:
def streetExists(streetName):
    if streetName in streetsDictionary:
        id = streetsDictionary[streetName]
    else:
        id = -1 # la calle no está mapeada
    return id

In [9]:
# adjacencyList: arreglo de arreglos, donde la pos del primer elemento (1er arreglo)
# es la calle y el arreglo son todas las calles con las que se intersecta
def createAdjacencyList(adjacencyList):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        for row in reader:  
            intersections = []
            for i in range(len(row)):
                if i != 0:
                    intersections.append(streetExists(row[i].lower()))
            adjacencyList.append(intersections)

    return adjacencyList

In [10]:
# creal la lista de adyacencia
intersectionsAdList = createAdjacencyList([])

##**Lista de Adyacencia de intersecciones (Grafo Principal)**

In [119]:
def createKeys(graph):
    intersectionsIDs = {}

    for i in range(len(graph)):
        for street in graph[i]:
            # si las keys aún no han sido creadas, las agrega al diccionario
            if (i, street) not in intersectionsIDs and street != -1:
                intersectionsIDs[(i, street)] = []

    return intersectionsIDs

In [120]:
def addIntersections(graph, i, j):  
    street = graph[i][j]

    if street != -1:
        # agrego la intersección már próxima hacia la izquierda
        if j > 0 and graph[i][j - 1] != -1:
            tup = (i, graph[i][j - 1])
            map[(i, street)].append(tup)

        # agrego la intersección már próxima hacia la derecha
        if j < len(graph[i]) - 1 and graph[i][j + 1] != -1:
            tup = (i, graph[i][j + 1])
            map[(i, street)].append(tup)

In [121]:
def addIntersectionsReversed(graph, i, j):  
    street = graph[i][j]

    # ahora voy al reverso de la lista
    if i != -1 and street != -1:
        if (street < len(graph)) and (i in graph[street]):
            tup = (street, i)
            map[(i, street)].append(tup)
            
            k = graph[street].index(i) # k = 2

            if k > 0 and graph[street][k - 1] != -1:
                tup = (street, graph[street][k - 1])
                map[(i, street)].append(tup)
        
            if k < len(graph[street]) - 1 and graph[street][k + 1] != -1:
                tup = (street, graph[street][k + 1])
                map[(i, street)].append(tup)

In [122]:
def joinIntersections(graph):
    n = len(graph)
    for i in range(n):        
        for j in range(len(graph[i])):
            addIntersections(graph, i, j)
            addIntersectionsReversed(graph, i, j)

In [123]:
# calcula el total de nodos solo par el grafo "map"
def totalNodes(graph):
    total = 0 
    for key, value in graph.items():
        total += len(value)
    return total

In [127]:
# grafo principal
map = createKeys(intersectionsAdList)
joinIntersections(intersectionsAdList)
#printDictionary(map)

##**Agregar valores aleatorios**
Agregar valores aleatorios al grafo principal (`map`). Los valores random fueron generados con Perlin Noise

In [70]:
def getGraphSize(graph):
    return len(graph)

In [151]:
def perlinNoise():
    # tamaño de cada de arreglo
    xpix = 10
    # cantidad de arreglos
    ypix = getGraphSize(map)
    noise = PerlinNoise(octaves=5, seed=1)
    pic = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]

    return np.array(np.array(pic)*100, dtype=int)

randValues = perlinNoise()

In [140]:
def sameNode(n1, n2):
    x1, y1 = n1
    x2, y2 = n2

    if (x1 == y2) and (y1 == x2):
        return True
    return False

In [141]:
def getListRandValues(n):
    r = random.randrange(0, n)
    
    while sum(randValues[r]) == 0:
        r = random.randrange(0, n)

    return list(randValues[r])

In [137]:
def getPositive(num):
    if num < 0:
        return num * -1
    else:
        return num

In [ ]:
# agrega un valor random (generado con perlin noise) al diccionario de lat y longs
def addRandValue(dictionary):

    randv = getListRandValues(len(latlong))

    for key, value in map.items():
        l = []
        for u in value:
            if sameNode(key, u):
                u += (0,)      
                l.append(u)
                continue

            if len(randv) > 0:
                u += (getPositive(randv.pop()),)
            else:
                randv = getListRandValues(len(latlong))
                u += (getPositive(randv.pop()),)
            l.append(u)
        dictionary[key] = l
    return dictionary
    
m = addRandValue({})

##**Diccionario de Latitudes y Longitudes para cada intersección**

In [17]:
def formatearLatitudeorLongitude(input):
    separar = input
    separar = separar.split(".");

    union= separar[0]+"."+separar[1]+separar[2];
    variable_float = float(union)

    return variable_float

In [94]:
def createLatLongDictionary(latlong):
    with open("Latitud y Longitud de calles.csv") as file: 
        reader = csv.reader(file)      
        # ignora el header (la 1era fila del csv)
        next(reader)
        for row in reader:
            # busco el id de la calle en mi diccionario de calles 
            intersection = streetExists(row[0]), streetExists(row[1])
            if intersection in map:
                latlong[intersection] = formatearLatitudeorLongitude(row[2]), formatearLatitudeorLongitude(row[3])
    return latlong
      
latlong = createLatLongDictionary({})

##**Escribir archivos**

In [ ]:
def writeNodes():
    with open("nodes.txt", "w") as f:
        with open('Calles de San Martín de Porres.csv') as file: 
            reader = csv.reader(file)

            for row in reader:
                for i in range(1, len(row)):                    
                    f.write(f"{row[0]}, {row[i]}\n")
writeNodes()

In [ ]:
# Streets Dictionary
def writeStreetsDictionary():
    with open("streetNames.txt", "w") as f:
        for key, value in streetsDictionary.items():
            f.write(key + "\n")
writeStreetsDictionary()

In [ ]:
# Intersections Adjacency List
def writeStreetsAdList():
    with open("streetsAdList.txt", "w") as f:
        for streets in intersectionsAdList:
            for i in range(len(streets)):
                streets[i] = str(streets[i])    
            f.write(', '.join(streets) + "\n")        
writeStreetsAdList()

In [ ]:
# Write street map
def writeStreetMap():
    with open("map.txt", "w") as f:  
        for key, value in map.items():
            listWOCommas = (str(value).strip('[]')).replace(",", "")
            f.write(str(key) + " " + listWOCommas + '\n')
writeStreetMap()

##**Aristas**

In [ ]:
# devuelve el tiempo actual automáticamente
def getCurrentTime():

    now = datetime.now()
    time = now.strftime("%H %p")
    time = time.split(" ")

    # convierto la hora a la timezone de Perú
    if time[0] <= "5":
        time[0] = 24 - (5 - int(time[0]))
    else:
        time[0] = int(time[0]) - 5
    
    return str(time[0])

def askTime():
    time = input("Digite la hora en formato de 24hrs: ");
    if int(time) not in range(0, 24):
        time = askTime() 
    return time

In [20]:
def getlongitude(n1, n2):
    lon1,lat1=n1
    lon2,lat2=n2
    if lon1 == lat2 and lon2 == lat1:
      longitude = 0
    else:
      lon1 = radians(lon1)
      lon2 = radians(lon2)
      lat1 = radians(lat1)
      lat2 = radians(lat2)
      dlon = lon2 - lon1
      dlat = lat2 - lat1
      a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
      c = 2 * asin(sqrt(a)) 
      r = 6371
      longitude = c * r
    
    return longitude

In [ ]:
def getFactor(time):
    factor = 0
    traffic_diccionary={
        "poco_trafico": 2,
        "trafico_promedio":4,
        "mucho_trafico":8
    }
    traffic_time = int(time)
    if(traffic_time in range (0,6) or traffic_time in range (19,25) ):
       factor = traffic_diccionary["poco_trafico"];

    elif(traffic_time in range (12,19)):
        factor = traffic_diccionary["trafico_promedio"];

    elif(traffic_time in range (6,10) or traffic_time in range (9,12)):
          factor = traffic_diccionary["mucho_trafico"];

    return factor;

In [ ]:
def getFactor(time, val):
    
    factor = 0
    #Puntajes van de 0 - 20
    
    # Puntaje de 0 - 5am
    if time in range(0, 6):
        for i in range(1, 3):
            print(val / i)

    # Puntaje de 6 - 10am
    if time in range(6, 11):
        range(13, 17)

    # Puntaje de 11 - 3pm
    if time in range(11, 16):
        range(3, 6)

    # Puntaje de 4 - 5pm
    if time in range(16, 18):
        range(10, 13)

    # Puntaje de 6 - 8pm
    if time in range(18, 21):
        range(17, 21)

    # Puntaje de 9 - 12pm
    if time in range(21, 25):
        range(6, 10)
    
    return factor

In [ ]:
def calculateWeight(n1, n2, time, val):
    
    longitude = getlongitude(n1, n2) * 100
    factor = getFactor(time)
    
    longdifference = 100 - longitude
    longPoints = longitude + ((longdifference * longitude) / 100)

    weight = 0
    if longitude > 0:
        weight = round(factor + longitude, 2)
    
    if (weight > 100):
        return 99
    else:
        return weight

In [ ]:
# el nuevo grafo será un diccionario, donde cada "key" contiene 1 arreglo de arreglos
# cada arreglo dentro del principal, tiene solo 2 posiciones
# 1era posición del arreglo: 1 tuple de intersecciones
# 2da posición del arreglo: peso de la arista con respecto al valor de la "key"

def updateGraph():
    time = askTime()

    # map con pesos
    mapW = {}
    i = 0
    j = 0
    for key in map:
        mapW[key] = []
        for arista in map[key]:
            val = valores[i][j]
            if key in latlong and arista in latlong:
                weight = calculateWeight(latlong[key], latlong[arista], time, val)
                mapW[key].append([arista, weight])
            j += 1
        j = 0
        i += 1
    return mapW

printDictionary(updateGraph())